# Module 11 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID. Make sure you submit both an .ipynb and .html version of your *completed* notebook. You can generate the HTML version using:

> ipython nbconvert [notebookname].ipynb

or use the File menu.

Add whatever additional imports you require here. Stick with the standard libraries and those required by the class. The import
gives you access to these functions: http://ipython.org/ipython-doc/stable/api/generated/IPython.core.display.html (Copy this link)
Which, among other things, will permit you to display HTML as the result of evaluated code (see HTML() or display_html()).

In [ ]:
from __future__ import division # so that 1/2 = 0.5 and not 0
from IPython.core.display import *

## Decision Trees

For this assignment you will be implementing and evaluating a Decision Tree using the ID3 Algorithm (**no** pruning or normalized information gain). Use the provided pseudocode. The data is located at (copy link):

http://archive.ics.uci.edu/ml/datasets/Mushroom

You can download the two files and read them to find out the attributes, attribute values and class labels as well as their locations in the file.

One of the things we did not talk about in the lectures was how to deal with missing values. In C4.5, missing values were handled by treating "?" as an implicit attribute value for every feature. For example, if the attribute was "size" then the domain would be ["small", "medium", "large", "?"]. Another approach is to skip instances with missing values. Yet another approach is to infer the missing value conditioned on the class. For example, if the class is "safe" and the color is missing, then we would infer the attribute value that is most often associated with "safe", perhaps "red". **Use the "?" approach for this assignment.**

As we did with the neural network, you should randomize your data (always randomize your data...you don't know if it is in some particular order like date of collection, by class label, etc.) and split it into two (2) sets. Train on the first set then test on the second set. Then train on the second set and test on the first set.

For regression, we almost always use something like Mean Squared Error to judge the performance of a model. For classification, there are a lot more options but for this assignment we will just look at classification error:

$$error\_rate=\frac{errors}{n}$$

You must implement four functions. `train` takes training_data and returns the Decision Tree as a data structure or object (for this one, I'm removing the OOP restriction...people often feel more comfortable writing a Tree in an OOP fashion). Make sure your Tree can be represented somehow.

```
def train( training_data):
   # returns a decision tree data structure
```

and `view` takes a tree and prints it out:

```
def view( tree):
    pass # probably doesn't return anything.
```

the purpose of the function is to be able to see what the tree looks like. It should be legible/pretty. You can use ASCII if you like or use something like NetworkX.

and `classify` takes a tree and a List of instances (possibly just one) and returns the classifications:

```
def classify( tree, test_data):
    # returns a list of classifications
```

and `evaluate` takes the classifications and the test_data and returns the error rate:

```
def evaluate( test_data, classifications):
    # returns an error rate
```

Basically, you're going to:

1. learn the tree for set 1
2. view the tree
3. classify set 2
4. evaluate the tree
5. learn the tree for set 2
6. view the tree
7. classify set 1
8. evalute the tree
9. average the classification error.

This is all that is required for this assignment. I'm leaving more of the particulars up to you but you can definitely use the last module as a guide.

**This is a very important assignment to reflect on the use of deepcopy because it has a natural recursive implementation**

-----

comment #1

In [ ]:
def helper1():
    pass

-----

Put your final function invocations starting here, one per cell: